In [288]:
import numpy as np
import pandas as pd
col_types = {u'id': np.int, 
             u'query': np.str, 
             u'product_title': np.str,
             u'product_description': np.str, 
             u'relevance': np.int, 
             u'weight': np.float,
             u'median_relevance': np.int, 
             u'relevance_variance': np.float,
             u'q_len' : np.int,
             u't_len' : np.int,
             u'd_len' : np.int,
             u'd_loglen' : np.float,
             u'd_exist' : np.int,
             u'q_words' : np.int,
             u'd_words' : np.int,
             u't_words' : np.int,
            }
train_in = pd.read_csv('clean_train.csv', dtype=col_types)
train_in = train_in.fillna ("")

test  = pd.read_csv('clean_test.csv', dtype=col_types)
test = test.fillna ("")

restored_relevance = pd.read_csv('restored_relevance_train.csv', dtype=col_types)

Делим обучающую выборку на train и validate

In [154]:
# split for train and test
import numpy as np
import random

rows_ids  = random.sample (train_in.id.unique(), int(train_in.id.nunique()*0.1))
validate  = train_in [train_in.id.isin (rows_ids)].copy()
train     = train_in [~train_in.id.isin (rows_ids)].copy()

В качестве классификатора используем регрессор на случайном лесе.<br>
- Регрессор выбран исходя из понимания упорядоченности оценок,
- а рендом форест - как один из немногих алгоритмов в scikit learn, поддерживающий веса фич

In [279]:
from sklearn.ensemble import RandomForestRegressor

predictor   = RandomForestRegressor(
                                    n_estimators=1000, 
                                    min_samples_split=30, 
                                    min_samples_leaf=16, 
                                    max_depth=10, 
                                    random_state=None, 
                                    max_features=None, 
                                    verbose=0, 
                                    max_leaf_nodes=None,
                                    n_jobs = -1
                                    )

Расстояние между оценками 4 и 3 может отличаться в реальности от расстояния <br>
между 1 и 2 <br>
или 2 и 3.

Поэтому расстояния между ответами подбирались вручную

In [280]:
class2regression = {
    1:-4,
    2:-1,
    3:1,
    4:4
    }

Ответ регрессии можно перевести в классы, разделив на 4 диапазона, <br>
чтобы количество ответов в каждом было пропорционально <br>
числу примеров соответствующего класса в обучающей выборке

In [284]:
# Считаем квантили частот оценок, чтобы переводить по ним результаты регрессии обратно в классы
train_qunantiles = [0,
                    1. * len (train_in [train_in.median_relevance <=1]) / len (train_in),
                    1. * len (train_in [train_in.median_relevance <=2]) / len (train_in),
                    1. * len (train_in [train_in.median_relevance <=3]) / len (train_in),
                    1]
print train_qunantiles

[0, 0.07619610159480213, 0.22150029533372712, 0.3924985233313644, 1]


Метод выбрасывает из данных все лишние столбцы и превращает матрицу, пригодную для классификации

In [248]:
def dataset2features (df):
    useless_columns = {'id',
                       'median_relevance',
                     'relevance_variance',
                     'query',
                     'product_title',
                     'product_description',
                     'product_description_clauses',
                     'relevance',
                     'weight',
                     'Unnamed: 0'}.intersection (df.columns)
    return  df.drop(list (useless_columns), axis=1).as_matrix()

Добавляем восстановленные метки классов в качестве целевых функций

In [ ]:
relevance_train = train.merge (restored_relevance, on="id")

Веса

In [287]:
np_weights = np.ndarray (buffer = np.array(relevance_train["weight"]), shape = len(relevance_train))

Обучаем предиктор

In [281]:
predictor.fit(dataset2features(relevance_train), 
              relevance_train["relevance"].apply (lambda x: class2regression[x]), 
              np_weights)

Предсказываем результаты

In [277]:
train_prediction  = predictor.predict (dataset2features(train))
train_prediction = list(pd.qcut(train_prediction, train_qunantiles, labels=[1,2,3,4]))

validate_prediction = predictor.predict (dataset2features(validate))
validate_prediction = list(pd.qcut(validate_prediction, train_qunantiles, labels=[1,2,3,4]))

test_prediction = predictor.predict (dataset2features(test))
test_prediction = list(pd.qcut(test_prediction, train_qunantiles, labels=[1,2,3,4]))

Оцениваем качество предсказания. Оно получилось завышенным по сравнению с результатами кагла

In [282]:
import utils
print "train kappa:   ", utils.quadratic_weighted_kappa (list(train["median_relevance"]),    train_prediction)
print "validate kappa:", utils.quadratic_weighted_kappa (list(validate["median_relevance"]), validate_prediction)

Restored relevance
train kappa:    0.771919342381
validate kappa: 0.675886605114


In [283]:
import pickle
pickle.dump(predictor, open("main_predictor.pkl",'w'))

Сохраняем предсказание

In [221]:
submission = pd.DataFrame({"id": test["id"], 
                           "prediction": test_prediction})
submission.to_csv("results.csv", index=False)